# Lab3 Intro
### Using RAG technique to enhance the solution by retrieving and injecting few shot examples into the prompt
### What is RAG?
RAG is a technique for augmenting LLM (Large Language Model) knowledge with additional data.
Large Language Models (LLMs) can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

A typical RAG application has two main components:

- **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

- **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

### What is Few-Shot Learning?
Few-shot learning is a machine learning paradigm that enables models to generalize from a limited number of labeled examples. Unlike traditional supervised learning, which requires large datasets for training, few-shot learning allows models to adapt to new tasks or classes with minimal data. This is akin to how humans can learn new concepts quickly from just a few instances.

### Using RAG and Few-Shot Learning to Classify VoC
With the introduction of RAG and Few-Shot Learning, we can now apply these concepts practically. In this lab, we will utilize the RAG technique together with Few-Shot Learning to classify Voice of Customer data. This session will build upon the skills developed in the last two labs, as we will continue to employ prompt engineering and embedding techniques. 

### Your objectives are:
- Setup a vector database using [Chroma integration in LangChain](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) 
- Use Amazon Titan embedding model to embed the labeled examples and store into the vector database
- Build a RAG chain that retrieves top k most relevant examples from vector store and generate the classification result

## 1. Install dependencies
- If you experince "ERROR: pip's dependency resolver does not currently...", please just ignore it

In [ ]:
!pip install -q langchain==0.2.16 langchain_aws==0.1.17 pandas==2.2.2 openpyxl==3.1.5 chromadb==0.5.5 langchain-chroma==0.1.2 python-dotenv==1.0.1

## 2. Initialize Bedrock model using Langchain

We will utilize the same approach used in Lab2 to setup the embedding object. 
- We use [Langchain](https://www.langchain.com/) SDK to build the application
- Initialize a BedrockEmbeddings object with 'Titan Text Embeddings V2" with the model id "amazon.titan-embed-text-v2:0"

In [ ]:
import boto3
import json
import copy
import pandas as pd
from termcolor import colored
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
bedrock_embedding = BedrockEmbeddings(model_id='amazon.titan-embed-text-v2:0')

- test run and preview the result

In [ ]:
test_embedding = bedrock_embedding.embed_documents(['I love programing'])
print(f"The embedding dimension is {len(test_embedding[0])}, first 10 elements are: {test_embedding[0][:10]}")

## 3. Preparing the Dataset to be classified. 
- The data (Voice of Customers) is subject to experiment usage only

We will once again load the categories.csv and comments.csv data files. In addition, we will introduce a new dataset, examples_with_label.csv, in this session. This new dataset provides an augmented knowledge base for the LLM, equipping it with relevant references. By incorporating these labeled examples into our workflow, it is expected to enhance the model's understanding and improve the accuracy of its output.

- Import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
category_definition = "data/categories.csv"
categories = pd.read_csv(category_definition)
display(categories)

### Load the customer review data

In [ ]:
comments_filepath = "data/comments.csv"
comments = pd.read_csv(comments_filepath)
comments

### Load examples data as we will use them as few shot examples

In [ ]:
examples_filepath = "data/examples_with_label.csv"
examples_df = pd.read_csv(examples_filepath) 
examples_df

## 4. Create Vector database to store the embedding vectors of customer review text

### 4.1 We will setup a vector database using [Chroma integration in LangChain](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) 
- Chroma is the AI-native open-source vector database. This is lightweight vector database suitable for developers to quickly develop and experiment with applications.
- We will use Chroma for our workshop experiment this time, but when you will develop your own application, there are variety of selections, such as Amazon OpenSearch, Amazon Aurora, Pinecone, Milvus,Faiss, etc. 

In [ ]:
from langchain_chroma import Chroma
# You can uncomment below code to reset the vector db, if you want to retry more times
# vector_store.reset_collection()  
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=bedrock_embedding,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

#### Let's consider each customer comment as an individual document, and then use the Langchain-Chroma SDK to upload all these customer comments into the vector store.

In [ ]:
from uuid import uuid4
import hashlib
from langchain_core.documents import Document

#### build langchain documents

In [ ]:
documents = []
for comment,groundtruth in examples_df.values:
    documents.append(
        Document(
        page_content=comment,
        metadata={"groundtruth":groundtruth}
        )
    )

#### add documents to vector store

In [ ]:
hash_ids = [hashlib.md5(doc.page_content.encode()).hexdigest() for doc in documents]
vector_store.add_documents(documents=documents, ids=hash_ids)


### 4.2 We will use vector search to compare the similarity between them and choose the category that is the similarest to the customer review text

Once the embedded customer review texts are stored in the vector database, we can apply the techniques we explored in Lab 2, semantic similarity search, to identify top k semantically relevant categorized review texts for each customer comment. 

- Test run Similarity search in vector store
- Performing a simple similarity search with relevance score can be done as follows:

In [ ]:
query = comments['comment'].sample(1).values[0]
print(colored(f"******query*****:\n{query}","blue"))

results = vector_store.similarity_search_with_relevance_scores(query, k=4)
print(colored("\n\n******results*****","green"))
for res, score in results:
    print(colored(f"* [SIM={score:3f}] \n{res.page_content}\n{res.metadata}","green"))

## 5. Build a RAG chain to retrieve only top K relevant examples for classification

#### 5.1 Customzie a LangChain Chat Model Class
- As the time of this event, LangChain has not supported the latest Amazon Foundation Model yet, we will customize a LangChain Chat Model Class, so that the latest model can be integrated with the chain prompting in LangChain.

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
import dotenv
import os
dotenv.load_dotenv()

from typing import Any, AsyncIterator, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage,AIMessage,SystemMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from langchain_core.runnables import run_in_executor
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser,XMLOutputParser
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate


class ChatModelOly(BaseChatModel):

    model_name: str
    br_runtime : Any = None
    ak: str = None
    sk: str = None
    region:str = None

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:

        if not self.br_runtime:
            if self.ak and self.sk:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime',
                                               region_name = self.region,
                                              aws_access_key_id = self.ak,
                                               aws_secret_access_key = self.sk
            
                                              )

            else:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime')
            
        
        new_messages = []
        system_message = ''
        for msg in messages:
            if isinstance(msg,SystemMessage):
                system_message = msg.content
            elif isinstance(msg,HumanMessage):
                new_messages.append({
                        "role": "user",
                        "content": [ {"text": msg.content}]
                    })
            elif isinstance(msg,AIMessage):
                new_messages.append({
                        "role": "assistant",
                        "content": [ {"text": msg.content}]
                    })

        
        temperature = kwargs.get('temperature',0.1)
        maxTokens = kwargs.get('max_tokens',3000)

        #Base inference parameters to use.
        inference_config = {"temperature": temperature,"maxTokens":maxTokens}


        # Send the message.
        response = self.br_runtime.converse(
            modelId=self.model_name,
            messages=new_messages,
            system=[{"text" : system_message}] if system_message else [],
            inferenceConfig=inference_config
        )
        output_message = response['output']['message']

        message = AIMessage(
            content=output_message['content'][0]['text'],
            additional_kwargs={},  # Used to add additional payload (e.g., function calling request)
            response_metadata={  # Use for response metadata
                **response['usage']
            },
        )
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])


    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        if not self.br_runtime:
            if self.ak and self.sk:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime',
                                               region_name = self.region,
                                              aws_access_key_id = self.ak,
                                               aws_secret_access_key = self.sk
            
                                              )

            else:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime')
            
        
        new_messages = []
        system_message = ''
        for msg in messages:
            if isinstance(msg,SystemMessage):
                system_message = msg.content
            elif isinstance(msg,HumanMessage):
                new_messages.append({
                        "role": "user",
                        "content": [ {"text": msg.content}]
                    })
            elif isinstance(msg,AIMessage):
                new_messages.append({
                        "role": "assistant",
                        "content": [ {"text": msg.content}]
                    })

        
        temperature = kwargs.get('temperature',0.1)
        maxTokens = kwargs.get('max_tokens',3000)

        #Base inference parameters to use.
        inference_config = {"temperature": temperature,"maxTokens":maxTokens}

        # Send the message.
        streaming_response = self.br_runtime.converse_stream(
            modelId=self.model_name,
            messages=new_messages,
            system=[{"text" : system_message}] if system_message else [],
            inferenceConfig=inference_config
        )
        # Extract and print the streamed response text in real-time.
        for event in streaming_response["stream"]:
            if "contentBlockDelta" in event:
                text = event["contentBlockDelta"]["delta"]["text"]
                # print(text, end="")
                chunk = ChatGenerationChunk(message=AIMessageChunk(content=[{"type":"text","text":text}]))

                if run_manager:
                    # This is optional in newer versions of LangChain
                    # The on_llm_new_token will be called automatically
                    run_manager.on_llm_new_token(text, chunk=chunk)

                yield chunk
            if 'metadata' in event:
                metadata = event['metadata']
                # Let's add some other information (e.g., response metadata)
                chunk = ChatGenerationChunk(
                    message=AIMessageChunk(content="", response_metadata={**metadata})
                )
                if run_manager:
                    run_manager.on_llm_new_token(text, chunk=chunk)
                yield chunk

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "echoing-chat-model-advanced"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        This information is used by the LangChain callback system, which
        is used for tracing purposes make it possible to monitor LLMs.
        """
        return {
            "model_name": self.model_name,
        }

llm = ChatModelOly(model_name="amazon.olympus-pro-v1:0")

- Define system prompt and user prompt template

The system prompt template has been copied over from Lab 1, while the user prompt has been modified slightly. For this user prompt, we add a part where we provide examples to the LLM. This is a demonstration for Few-Shot Learning. The model will takes these exmaples as references when making classifications.

In [ ]:
system = """You are a professional  customer feedback analyst. Your daily task is to categorize user feedback.
You will be given an input in the form of a JSON array. Each object in the array contains a comment ID and a 'c' field representing the user's comment content.
Your role is to analyze these comments and categorize them appropriately.
Please note:
1. Only output valid XML format data.
2. Do not include any explanations or additional text outside the XML structure.
3. Ensure your categorization is accurate and consistent.
4. If you encounter any ambiguous cases, use your best judgment based on the context provided.
5. Maintain a professional and neutral tone in your categorizations.
"""

user = """
Please categorize user comments according to the following category tags library:
<categories>
{tags}
</categories>

Here are examples for your to categorize:
<examples>
{examples}
<examples>

Please follow these instructions for categorization:
<instruction>
1. Categorize each comment using the tags above. If no tags apply, output "Invalid Data".
2. Summarize the comment content in no more than 50 words. Replace any double quotation marks with single quotation marks.
</instruction>

Below are the customer comments records to be categorized. The input is an array, where each element has an 'id' field representing the complaint ID and a 'c' field summarizing the complaint content.
<comments>
{input}
</comments>

For each record, summarize the comment, categorize according to the category explainations, and return the  ID, summary , reasons for tag matches, and category.

Output format example:
<output>
  <item>
    <id>xxx</id>
    <summary>xxx</summary>
    <reason>xxx</reason>
    <category>xxx</category>
  </item>
</output>

Skip the preamble and output only valid XML format data. Remember:
- Avoid double quotation marks within quotation marks. Use single quotation marks instead.
- Replace any double quotation marks in the content with single quotation marks.
"""

#### 5.2 Define a RAG pipeline with LangChain

We are now set to create a RAG pipeline using Langchain. The workflow of this pipelines involves:
- initially, it retrieves the top k most relevant customer review texts, which are labeled with categories, from the vector database. 
- This retrieved information is then integrated into the prompt. 
- Invoke LLM to generate outputs according to the provided instructions. 
- Finally, the output is displayed. 

- Define langchain retriever, use `k = 5` to retrive top 5 records

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
def format_docs(docs):
    formatted = "\n".join([f"<content>{doc.page_content}</content>\n<cateogry>{doc.metadata['groundtruth']}</cateogry>" for doc in docs])
    # print(colored(f"*****retrived examples******:\n{formatted}","yellow"))
    return formatted

- test run for retriever

In [ ]:
retrieved_docs = (retriever|format_docs).invoke("my phone always freeze")

- Define prompt template

In [ ]:
prompt = ChatPromptTemplate([
    ('system',system),
    ('user',user)],
partial_variables={'tags':categories['mappings'].values}
)

- Define a RAG prompt chain

In [ ]:
chain = (
    {"examples":retriever|format_docs,"input":RunnablePassthrough()}
    | prompt
    | llm
    | XMLOutputParser()
)

- Convert comments into a data array for iterative processing.

In [ ]:
sample_data = [str({"id":'s_'+str(i),"comment":x[0]}) for i,x in enumerate(comments.values)]
print(sample_data[:3])

### We will iterate through each comment for categorization.
* This step will take around 4~5 mins to compelete
* <span style="color: red;">Hint!</span> Due to the uncertainly of LLM generation, the JSON output might fail occasionally, if you happen to experience error such as `KeyError: 'output'`, please re-run the cell again

In [ ]:
%%time
import math,json
from json import JSONDecodeError

resps = []


def retry_call(chain,args: Dict[str,Any],times:int=3):
    """
      Retry mechanism to ensure the success rate of final structure output 
    """
    try:
        content = chain.invoke(args)
        if 'output' not in content:
            raise (JSONDecodeError('KeyError: output'))
        return content
    except Exception as e:
        if times:
            print(f'Exception, return [{times}]')
            return retry_call(chain,args,times=times-1)
        else:
            raise(JSONDecodeError(e))

for i in range(len(sample_data)):
    data = sample_data[i]
    # print(colored(f"*****input[{i}]*****:\n{data}","blue"))

    # resp = chain.invoke(data)
    resp = retry_call(chain,data)
    print(colored(f"*****response*****\n{resp}","green"))
    # resps += json.loads(resp)
    for item in resp['output']:
        row={}
        for it in item['item']:
            row[list(it.keys())[0]]=list(it.values())[0]
        resps.append(row)

### Check if all the data has been processed correctly

In [ ]:
assert len(resps) == len(sample_data), "Due to the uncertainly of LLM generation, the JSON output might fail occasionally, if you happen to experience error, please re-run above cell again"

- covert the data array to pandas dataframe

In [ ]:
prediction_df = pd.DataFrame(resps).rename(columns={"category":"predict_label"}).drop_duplicates(['id']).reset_index(drop='index')
# convert the label value to lowercase
prediction_df['predict_label'] = prediction_df['predict_label'].apply(lambda x: x.strip().lower().replace("'",""))
prediction_df

### Merge the prediction result to ground truth dataframe

- copy comments to ground_truth dataframe
- merge the date prediction to the groudtruth data

In [ ]:
ground_truth = comments.copy()
# convert the label value to lowercase
ground_truth['groundtruth'] = ground_truth['groundtruth'].apply(lambda x: x.strip().lower())
merge_df=pd.concat([ground_truth,prediction_df],axis=1)
merge_df

## 6.Calculate the accuracy

In [ ]:
accuracy = (merge_df['groundtruth'] == merge_df['predict_label']).mean()

In [ ]:
print(colored(f"****accuracy:****\n{accuracy}","green"))

In [ ]:
### save the result 
merge_df.to_csv('result_lab_3.csv',index=False)